# GeekBrains, ML in Business
# Lesson 4 Homework

Ссылки:
* https://towardsdatascience.com/a-quick-uplift-modeling-introduction-6e14de32bfe0
* https://habr.com/ru/company/ru_mts/blog/485980/#reference1
* https://en.wikipedia.org/wiki/Uplift_modelling
* https://www.youtube.com/watch?v=yFQAIJBYXI0
* https://www.youtube.com/watch?v=jCUcYiBK03I
* https://www.uplift-modeling.com/en/latest/
* https://arxiv.org/pdf/1809.04559.pdf
* https://catboost.ai/docs/concepts/about.html

Библиотеки и пакеты:
* causalml
* sklift
* catboost

**Импорт библиотек**

In [2]:
import numpy as np
import pandas as pd

# import matplotlib.pyplot as plt
# import seaborn as sns

from sklearn.model_selection import train_test_split
#, cross_val_score
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, log_loss, confusion_matrix


# %matplotlib inline

**Функции и классы для заданий**

Пайплайны трансформаций.

In [7]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

# Объединяем пайплайны признаков в один
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)    

## Задание 1
Скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention.

### Решение Задания 1

Скачали данные, импортируем.

In [3]:
df = pd.read_csv('data.csv', delimiter=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        64000 non-null  int64  
 1   history        64000 non-null  float64
 2   used_discount  64000 non-null  int64  
 3   used_bogo      64000 non-null  int64  
 4   zip_code       64000 non-null  object 
 5   is_referral    64000 non-null  int64  
 6   channel        64000 non-null  object 
 7   offer          64000 non-null  object 
 8   conversion     64000 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 4.4+ MB


In [4]:
df.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


In [5]:
df['conversion'].value_counts()

0    54606
1     9394
Name: conversion, dtype: int64

In [6]:
df['offer'].value_counts()

Buy One Get One    21387
Discount           21307
No Offer           21306
Name: offer, dtype: int64

## Задание 2
Поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

### Решение Задания 2

Переименовываем.

In [7]:
df = df.rename(columns={'conversion': 'target', 'offer': 'treatment'})

Приводим в бинарный вид и удаляем признак treatment_No Offer - он не дает никакой дополнительной информации, т.к. нули в 2-х других признаках автоматически означают No Offer.

In [8]:
df = pd.get_dummies(df, columns=['treatment']).drop(columns=['treatment_No Offer'])

In [9]:
df

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,target,treatment_Buy One Get One,treatment_Discount
0,10,142.44,1,0,Surburban,0,Phone,0,1,0
1,6,329.08,1,1,Rural,1,Web,0,0,0
2,7,180.65,0,1,Surburban,1,Web,0,1,0
3,9,675.83,1,0,Rural,1,Web,0,0,1
4,2,45.34,1,0,Urban,0,Web,0,1,0
...,...,...,...,...,...,...,...,...,...,...
63995,10,105.54,1,0,Urban,0,Web,0,0,1
63996,5,38.91,0,1,Urban,1,Phone,0,0,1
63997,6,29.99,1,0,Urban,1,Phone,0,0,1
63998,1,552.94,1,0,Surburban,1,Multichannel,0,1,0


## Задание 3
Сделать разбиение набора данных не тренировочную и тестовую выборки.

### Решение Задания 3

Делим датасет.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['target']), 
                                                    df['target'],
                                                    random_state=0)

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((48000, 9), (16000, 9), (48000,), (16000,))

## Задание 4
Сделать feature engineering на ваше усмотрение (допускается свобода выбора методов).

### Решение Задания 4

In [22]:
df['recency'].value_counts()

1     8952
10    7565
2     7537
9     6441
3     5904
4     5077
6     4605
5     4510
7     4078
11    3504
8     3495
12    2332
Name: recency, dtype: int64

## Задание 5
Провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями.

### Решение Задания 5

Датафрейм для сравнения метрик.

In [ ]:
metrics_df = pd.DataFrame(columns=['precision', 'recall', 'f1_score', 'roc_auc', 'cv_roc_auc', 'cv_roc_auc_std'])

Модель.

Сохраняем метрики модели.

In [ ]:
metrics_model1 = validate_model(model1, X_train, X_test, y_train, y_test)
metrics_df = metrics_df.append(metrics_model1, ignore_index=True)

## Задание 6
В конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей.

### Решение Задания 6

In [ ]:
metrics_df

## Задание 7
Построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево.

### Решение Задания 7

## Задание 8*
Для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета (конверсии - поле conversion) от значения uplift: 1) сделать прогноз и получить uplift для тестовой выборки 2) отсортировать тестовую выборку по uplift по убыванию 3) разбить на децили (pandas qcut вам в помощь) 4) для каждого дециля посчитать среднюю conversion.

### Решение Задания 8*

## Задание 9*
Построить модель UpliftRandomForestClassifier и попытаться описать словами полученное дерево.

### Решение Задания 9*